In [5]:
#使用逻辑回归预测客户流失概率
import pandas as pd
import numpy as np
import statsmodels.api as sma 

inputCsv='churn.csv'
data=pd.read_csv(inputCsv)

#数据预处理
#将列标题的空格替换为下划线，将引号和问号去除，标题字母变为小写
data.columns=data.columns.str.replace(' ','_').str.replace('\'','').str.strip('?').str.lower()
#将churn字段值中'.'删除,
data.churn=data.churn.str.strip('.')
#print(data.head(5))
print(data.columns)
#新增一个字段，将churn字段转换为01编码字段
data['churn01']=np.where(data.churn=='True',1,0)
#对字段intl_plan及vmail_plan进行独热编码（新增虚拟变量）哑变量，prefix设置产生变量的名字
intl_plan_dummy=pd.get_dummies(data.intl_plan,prefix='intl_plan')
vmail_plan_dummy=pd.get_dummies(data.vmail_plan,prefix='vmail_plan')
#添加常数项及生成自变量和因变量
x_data=sma.add_constant(data[data.columns.difference(['intl_plan',
                                                          'vmail_plan',
                                                          'churn01',
                                                          'churn',
                                                          'state',
                                                          'phone',
                                                          'account_length',
                                                          'area_code'])].\
                          join(intl_plan_dummy.intl_plan_yes).join(vmail_plan_dummy.vmail_plan_yes))
print('****',x_data.columns)
y_data=data['churn01']


#将数据划分为训练集和测试集，训练集为第一行至倒数第10行，测试集为最后10行
x_Train=x_data.iloc[0:-10,:]
y_Train=y_data.iloc[0:-10]
x_Test=x_data.tail(10)
y_Test=y_data.tail(10)

#根据训练集训练获取模型参数
lr=sma.Logit(y_Train,x_Train)
result=lr.fit()
print(result.summary2())


#根据模型获取测试集结果
predicted=lr.fit().predict(x_Test)
compare=pd.DataFrame({'predictedValue':predicted,'actualValue':y_Test})
print(compare)


Index(['state', 'account_length', 'area_code', 'phone', 'intl_plan',
       'vmail_plan', 'vmail_message', 'day_mins', 'day_calls', 'day_charge',
       'eve_mins', 'eve_calls', 'eve_charge', 'night_mins', 'night_calls',
       'night_charge', 'intl_mins', 'intl_calls', 'intl_charge',
       'custserv_calls', 'churn'],
      dtype='object')
**** Index(['const', 'custserv_calls', 'day_calls', 'day_charge', 'day_mins',
       'eve_calls', 'eve_charge', 'eve_mins', 'intl_calls', 'intl_charge',
       'intl_mins', 'night_calls', 'night_charge', 'night_mins',
       'vmail_message', 'intl_plan_yes', 'vmail_plan_yes'],
      dtype='object')
Optimization terminated successfully.
         Current function value: 0.324232
         Iterations 7
                          Results: Logit
Model:              Logit            Pseudo R-squared: 0.217      
Dependent Variable: churn01          AIC:              2188.8469  
Date:               2019-11-27 09:09 BIC:              2292.6935  
No. Observati